# LABORATORIUM 5 - DRZEWO DECYZYJNE

### 5.1 Implementacja drzewa decyzyjnego

In [ ]:
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier as SkDecisionTree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons


#### Generowanie danych:

In [ ]:
def generate_data(n_samples, n_features, random_state):
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_features,
        n_redundant=0,
        n_clusters_per_class=1,
        random_state=random_state
    )
    # y = 2 * y - 1
    return X, y

def generate_moons(n_samples, random_state):
    X_moons, y_moons = make_moons(n_samples=n_samples, noise=0.1, random_state=random_state)
    # y_moons = 2 * y_moons - 1
    return X_moons, y_moons

def generate_blobs(n_samples, n_features, random_state):
    X_uni, y_uni = make_blobs(n_samples=n_samples, centers=2, n_features=n_features, random_state=random_state)
    # y_uni = 2 * y_uni - 1
    return X_uni, y_uni

#### Implementacja własnej funkcji do drzewa:

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
    
    def entropy(self, y):
        counts = np.bincount(y, minlength=self.n_classes)
        probs = counts / len(y)
        probs = probs[probs > 0] # usuwamy zera by uniknąć log(0)
        return -np.sum(probs * np.log2(probs))
    
    def information_gain(self, X, y, feature, threshold):
        parent_entropy = self.entropy(y)
        feature_data = X[:, feature]
        left_mask = feature_data <= threshold
        right_mask = ~left_mask

        if np.sum(left_mask) == 0 or np.sum(right_mask) == 0: ## jeśli nie ma podziału to zysk informacyjny jest zerowy
            return 0

        left_entropy = self.entropy(y[left_mask])
        right_entropy = self.entropy(y[right_mask])
        n = len(y)
        n_left, n_right = np.sum(left_mask), np.sum(right_mask)
        child_entropy = (n_left / n) * left_entropy + (n_right / n) * right_entropy
        return parent_entropy - child_entropy
    
    def find_best_threshold(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None

        for feature in range(self.n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self.information_gain(X, y, feature, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold
    
    def build_tree(self, X, y, depth):
        n_samples = len(y)
        n_labels = len(np.unique(y))

        # Warunki zatrzymania
        if (self.max_depth is not None and depth >= self.max_depth) or n_labels == 1 or n_samples < 2:
            leaf_value = Counter(y).most_common(1)[0][0]
            return Node(value=leaf_value)

        # Znajdź najlepszy podział
        feature, threshold = self.find_best_threshold(X, y)
        if feature is None:  # Brak poprawy
            leaf_value = Counter(y).most_common(1)[0][0]
            return Node(value=leaf_value)

        # Podział danych
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask
        left = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        right = self.build_tree(X[right_mask], y[right_mask], depth + 1)

        return Node(feature, threshold, left, right)
    